**Guardrails:** are important to prevent misuse and mitigate concerns on bad actors. Within sql-sidekick, we have added support for scanning malicious patterns that might occur in generated SELECT statements that might destroy the database or compromise sensitive information. 
We adopted a 2-step approach, using pattern matching and LLM prompting to ensure safety and compliance in a product setting. The self-corrective loop with external feedback helps maintain syntactical integrity.

_Reference:_
1. https://github.com/payloadbox/sql-injection-payload-list#generic-sql-injection-payloads
2. https://www.invicti.com/blog/web-security/sql-injection-cheat-sheet/#InlineSamples

In [ ]:
# !pip install --force-reinstall sql_sidekick-0.x.x-py3-none-any.whl

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sidekick import prompter
prompter.__version__

'0.2.0'

In [3]:
import os
import pandas as pd
from pathlib import Path


from sidekick.prompter import ask, db_setup
from sidekick.query import SQLGenerator
from sidekick.schema_generator import generate_schema
from sidekick.utils import setup_dir

import warnings
warnings.filterwarnings("ignore")

In [4]:
# env variables

os.environ['OPENAI_API_KEY'] = ""
os.environ['H2OGPT_URL'] = ''
os.environ['H2OGPT_API_TOKEN'] = ""
os.environ['H2OGPTE_URL'] = "https://h2ogpte.genai.h2o.ai"  # e.g. https://<>.h2ogpte.h2o.ai
os.environ['H2OGPTE_API_TOKEN'] = ""


base_path = "."
cache_path = f"{base_path}/var/lib/tmp"
setup_dir(base_path)

HOST_NAME = "localhost"
USER_NAME = "sqlite"
PASSWORD = "abc"
DB_NAME = "query_test"
PORT = "5432"

In [5]:
data_path = "./sleep_health_and_lifestyle_dataset.csv"
# Replace table_name as needed
table_name = "sleep_health_and_lifestyle"
table_schema, table_info_path = generate_schema(data_path=data_path, output_path=f"{cache_path}/{table_name}_table_info.jsonl")

# Initialize DB
if Path(f"{base_path}/db/sqlite/{DB_NAME}.db").exists():
    os.remove(f"{base_path}/db/sqlite/{DB_NAME}.db")

In [6]:
# Set add_sample=False if no need to add rows to the table
response , err = db_setup(
                db_name=DB_NAME,
                hostname=HOST_NAME,
                user_name=USER_NAME,
                password=PASSWORD,
                port=PORT,
                table_info_path=table_info_path,
                table_samples_path=data_path,
                table_name=table_name,
                local_base_path=base_path,
                add_sample=True
            )

 Information supplied:
 query_test, localhost, sqlite, abc, 5432


2024-01-19 22:24:26.138 | DEBUG    | sidekick.db_config:create_db:108 - SQLite DB is created successfully.


Database created successfully!
Table name: sleep_health_and_lifestyle


2024-01-19 22:24:26.142 | DEBUG    | sidekick.db_config:_extract_schema_info:162 - Using schema information from: ./var/lib/tmp/sleep_health_and_lifestyle_table_info.jsonl
2024-01-19 22:24:26.144 | DEBUG    | sidekick.db_config:create_table:186 - Schema info used for creating table:
 Person_ID NUMERIC,
Gender TEXT COLLATE NOCASE,
Age NUMERIC,
Occupation TEXT COLLATE NOCASE,
Sleep_Duration NUMERIC,
Quality_of_Sleep NUMERIC,
Physical_Activity_Level NUMERIC,
Stress_Level NUMERIC,
BMI_Category TEXT COLLATE NOCASE,
Blood_Pressure TEXT COLLATE NOCASE,
Heart_Rate NUMERIC,
Daily_Steps NUMERIC,
Sleep_Disorder TEXT COLLATE NOCASE
2024-01-19 22:24:26.155 | INFO     | sidekick.db_config:create_table:198 - Table created: sleep_health_and_lifestyle


Checked table sleep_health_and_lifestyle exists in the DB.


2024-01-19 22:24:26.159 | DEBUG    | sidekick.db_config:add_samples:222 - Adding sample values to table: ./sleep_health_and_lifestyle_dataset.csv
2024-01-19 22:24:26.164 | DEBUG    | sidekick.db_config:add_samples:228 - Inserting chunk: 0
2024-01-19 22:24:26.264 | INFO     | sidekick.db_config:add_samples:233 - Data inserted into table: sleep_health_and_lifestyle
2024-01-19 22:24:26.267 | INFO     | sidekick.db_config:add_samples:238 - Number of rows inserted: 374


Created a Database query_test. Inserted sample values from ./sleep_health_and_lifestyle_dataset.csv into table sleep_health_and_lifestyle, please ask questions!


## Example 1:

In [7]:
# Example lets consider a simple query as mentioned below,
# Expected output: a single row having information about person_id=105

input_q = """
    SELECT * FROM sleep_health_and_lifestyle WHERE person_id = 105;
    """

# We are running in Debug Mode as we are validating SQL statements directly `debug_mode=True`
result = None
question = f"Execute SQL:\n{input_q}"

result, _, error = ask(
    question=question,
    table_info_path=table_info_path,
    sample_queries_path=None,
    table_name=table_name,
    is_command=False,
    model_name="h2ogpt-sql-nsql-llama-2-7B",
    is_regenerate=False,
    is_regen_with_options=False,
    execute_query=True,
    guardrails=False,
    local_base_path=base_path,
    debug_mode=True,
    self_correction=False
)

2024-01-19 22:24:26.321 | INFO     | sidekick.prompter:ask:490 - Table in use: ['sleep_health_and_lifestyle']
2024-01-19 22:24:26.322 | INFO     | sidekick.prompter:ask:491 - SQL dialect for generation: sqlite
2024-01-19 22:24:26.323 | INFO     | sidekick.prompter:ask:524 - Setting context...
2024-01-19 22:24:26.324 | INFO     | sidekick.prompter:ask:525 - Question: Execute SQL:

    SELECT * FROM sleep_health_and_lifestyle WHERE person_id = 105;
    
2024-01-19 22:24:26.325 | DEBUG    | sidekick.prompter:ask:543 - Table info path: ./var/lib/tmp/sleep_health_and_lifestyle_table_info.jsonl
2024-01-19 22:24:26.326 | INFO     | sidekick.utils:is_resource_low:353 - Number of GPUs: 1
2024-01-19 22:24:26.327 | DEBUG    | sidekick.utils:is_resource_low:357 - Information on device: 0
2024-01-19 22:24:27.846 | INFO     | sidekick.utils:is_resource_low:360 - Total Memory: 23GB
2024-01-19 22:24:27.848 | INFO     | sidekick.utils:is_resource_low:361 - Free GPU memory: 21GB
2024-01-19 22:24:27.860 

The results are:
 [(105, 'Female', 36, 'Teacher', 7.2, 8, 60, 4, 'Normal', '115/75', 68, 7000, 'Sleep Apnea')]
Exiting...


In [8]:
# Injecting malicious tokens
# Example: OR 1=1
# Below user input is a syntactically valid SQL statement but since `0R 1=1` is always true, instead of returning information
# about person_id=105; a bad actor might get information about all user.

# Built-in guardrails is disabled
# Result contains more than one rows

result = None
input_q = """
    SELECT * FROM sleep_health_and_lifestyle WHERE person_id = 105 OR 1=1;
    """
question = f"Execute SQL:\n{input_q}"
#1. Self correction is disabled
result, _, error = ask(
    question=question,
    table_info_path=table_info_path,
    sample_queries_path=None,
    table_name=table_name,
    is_command=False,
    model_name="h2ogpt-sql-nsql-llama-2-7B",
    is_regenerate=False,
    is_regen_with_options=False,
    execute_query=True,
    guardrails=False,
    local_base_path=base_path,
    debug_mode=True,
    self_correction=False
)

# format final output
import ast

res = ast.literal_eval(result[4])
final_res = pd.DataFrame(res)

print(f"Number of rows returned: {final_res.shape}") 

2024-01-19 22:24:27.939 | INFO     | sidekick.prompter:ask:490 - Table in use: ['sleep_health_and_lifestyle']
2024-01-19 22:24:27.941 | INFO     | sidekick.prompter:ask:491 - SQL dialect for generation: sqlite
2024-01-19 22:24:27.942 | INFO     | sidekick.prompter:ask:524 - Setting context...
2024-01-19 22:24:27.942 | INFO     | sidekick.prompter:ask:525 - Question: Execute SQL:

    SELECT * FROM sleep_health_and_lifestyle WHERE person_id = 105 OR 1=1;
    
2024-01-19 22:24:27.943 | DEBUG    | sidekick.prompter:ask:543 - Table info path: ./var/lib/tmp/sleep_health_and_lifestyle_table_info.jsonl
2024-01-19 22:24:27.944 | INFO     | sidekick.utils:is_resource_low:353 - Number of GPUs: 1
2024-01-19 22:24:27.945 | DEBUG    | sidekick.utils:is_resource_low:357 - Information on device: 0
2024-01-19 22:24:27.946 | INFO     | sidekick.utils:is_resource_low:360 - Total Memory: 23GB
2024-01-19 22:24:27.947 | INFO     | sidekick.utils:is_resource_low:361 - Free GPU memory: 21GB
2024-01-19 22:24:

Preview of the result:
    Person_ID Gender  Age         Occupation  Sleep_Duration  Quality_of_Sleep  \
0          1   Male   27  Software Engineer             6.1                 6   
1          2   Male   28             Doctor             6.2                 6   

   Physical_Activity_Level  Stress_Level BMI_Category Blood_Pressure  \
0                       42             6   Overweight         126/83   
1                       60             8       Normal         125/80   

   Heart_Rate  Daily_Steps Sleep_Disorder  
0          77         4200           None  
1          75        10000           None  
Exiting...
Number of rows returned: (101, 13)


In [9]:
# With guardrails 'ON', we are able prevent such ambigious intent
# Enabling quardrails is configurable by setting the `guardrails=True/false`

result = None
input_q = """
    SELECT * FROM sleep_health_and_lifestyle WHERE person_id = 105 OR 1=1;
    """
question = f"Execute SQL:\n{input_q}"

result, _, error = ask(
    question=question,
    table_info_path=table_info_path,
    sample_queries_path=None,
    table_name=table_name,
    is_command=False,
    model_name="h2ogpt-sql-nsql-llama-2-7B",
    is_regenerate=False,
    is_regen_with_options=False,
    execute_query=True,
    guardrails=True,
    local_base_path=base_path,
    debug_mode=True,
    self_correction=False
)

2024-01-19 22:24:28.043 | INFO     | sidekick.prompter:ask:490 - Table in use: ['sleep_health_and_lifestyle']
2024-01-19 22:24:28.044 | INFO     | sidekick.prompter:ask:491 - SQL dialect for generation: sqlite
2024-01-19 22:24:28.046 | INFO     | sidekick.prompter:ask:524 - Setting context...
2024-01-19 22:24:28.047 | INFO     | sidekick.prompter:ask:525 - Question: Execute SQL:

    SELECT * FROM sleep_health_and_lifestyle WHERE person_id = 105 OR 1=1;
    
2024-01-19 22:24:28.048 | DEBUG    | sidekick.prompter:ask:543 - Table info path: ./var/lib/tmp/sleep_health_and_lifestyle_table_info.jsonl
2024-01-19 22:24:28.049 | INFO     | sidekick.utils:is_resource_low:353 - Number of GPUs: 1
2024-01-19 22:24:28.051 | DEBUG    | sidekick.utils:is_resource_low:357 - Information on device: 0
2024-01-19 22:24:28.052 | INFO     | sidekick.utils:is_resource_low:360 - Total Memory: 23GB
2024-01-19 22:24:28.053 | INFO     | sidekick.utils:is_resource_low:361 - Free GPU memory: 21GB
2024-01-19 22:24:

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.3.0-dev5`.
You can enable strict version checking by passing strict_version_check=True.
The results are:
 The input question has malicious patterns, **('or', '', ''), or 1=1** that could lead to SQL Injection.
Sorry, I will not be able to provide an answer.
Please try rephrasing the question. The query is vulnerable to SQL injection. The use of the OR clause with a constant value (1=1) is a common indicator of an injection attack. This suggests that an attacker may be able to manipulate the query to execute arbitrary SQL code, potentially leading to unauthorized access or modification of data.
Exiting...


## Example 2:
Classic Inline Comment SQL Injection Attack

In [10]:
result = None
input_q = """
    10; DROP TABLE sleep_health_and_lifestyle /*;
    """
question = f"Execute SQL:\n{input_q}"

result, _, error = ask(
    question=question,
    table_info_path=table_info_path,
    sample_queries_path=None,
    table_name=table_name,
    is_command=False,
    model_name="h2ogpt-sql-nsql-llama-2-7B",
    is_regenerate=False,
    is_regen_with_options=False,
    execute_query=True,
    guardrails=True,
    local_base_path=base_path,
    debug_mode=True,
    self_correction=False
)

2024-01-19 22:24:37.276 | INFO     | sidekick.prompter:ask:490 - Table in use: ['sleep_health_and_lifestyle']
2024-01-19 22:24:37.277 | INFO     | sidekick.prompter:ask:491 - SQL dialect for generation: sqlite
2024-01-19 22:24:37.278 | INFO     | sidekick.prompter:ask:524 - Setting context...
2024-01-19 22:24:37.279 | INFO     | sidekick.prompter:ask:525 - Question: Execute SQL:

    10; DROP TABLE sleep_health_and_lifestyle /*;
    
2024-01-19 22:24:37.280 | DEBUG    | sidekick.prompter:ask:543 - Table info path: ./var/lib/tmp/sleep_health_and_lifestyle_table_info.jsonl
2024-01-19 22:24:37.281 | INFO     | sidekick.utils:is_resource_low:353 - Number of GPUs: 1
2024-01-19 22:24:37.282 | DEBUG    | sidekick.utils:is_resource_low:357 - Information on device: 0
2024-01-19 22:24:37.283 | INFO     | sidekick.utils:is_resource_low:360 - Total Memory: 23GB
2024-01-19 22:24:37.284 | INFO     | sidekick.utils:is_resource_low:361 - Free GPU memory: 21GB
2024-01-19 22:24:37.294 | INFO     | sidek

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.3.0-dev5`.
You can enable strict version checking by passing strict_version_check=True.
The results are:
 The input question has malicious patterns, **SQL keywords does not start with SELECT, only SELECT queries are allowed.** that could lead to SQL Injection.
Sorry, I will not be able to provide an answer.
Please try rephrasing the question.
Exiting...


In [11]:
result = None
input_q = """
    SELECT /*!32302 1/0, */ 1 FROM sleep_health_and_lifestyle;
    """
question = f"Execute SQL:\n{input_q}"

result, _, error = ask(
    question=question,
    table_info_path=table_info_path,
    sample_queries_path=None,
    table_name=table_name,
    is_command=False,
    model_name="h2ogpt-sql-nsql-llama-2-7B",
    is_regenerate=False,
    is_regen_with_options=False,
    execute_query=True,
    guardrails=True,
    local_base_path=base_path,
    debug_mode=True,
    self_correction=False
)

2024-01-19 22:32:45.505 | INFO     | sidekick.prompter:ask:490 - Table in use: ['sleep_health_and_lifestyle']
2024-01-19 22:32:45.506 | INFO     | sidekick.prompter:ask:491 - SQL dialect for generation: sqlite
2024-01-19 22:32:45.508 | INFO     | sidekick.prompter:ask:524 - Setting context...
2024-01-19 22:32:45.509 | INFO     | sidekick.prompter:ask:525 - Question: Execute SQL:

    SELECT /*!32302 1/0, */ 1 FROM sleep_health_and_lifestyle;
    
2024-01-19 22:32:45.509 | DEBUG    | sidekick.prompter:ask:543 - Table info path: ./var/lib/tmp/sleep_health_and_lifestyle_table_info.jsonl
2024-01-19 22:32:45.510 | INFO     | sidekick.utils:is_resource_low:353 - Number of GPUs: 1
2024-01-19 22:32:45.511 | DEBUG    | sidekick.utils:is_resource_low:357 - Information on device: 0
2024-01-19 22:32:45.512 | INFO     | sidekick.utils:is_resource_low:360 - Total Memory: 23GB
2024-01-19 22:32:45.513 | INFO     | sidekick.utils:is_resource_low:361 - Free GPU memory: 21GB
2024-01-19 22:32:45.524 | INF

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.3.0-dev5`.
You can enable strict version checking by passing strict_version_check=True.
The results are:
 The input question has malicious patterns, **/*!32302 1/0, */** that could lead to SQL Injection.
Sorry, I will not be able to provide an answer.
Please try rephrasing the question. The query contains a comment with a SQL injection vulnerability. The comment `/*!32302 1/0, */` is not properly formatted and could be used to inject malicious SQL code.
Exiting...


**Note:**

Examples shown above are not an exhaustive list but a WIP.
We have differrent levels of guardrails enabled for safe execution but at the same time we are updating/improving them as we learn further.
Don't hesitate to reach out.